In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Read the files
#df_final_demo = pd.read_csv('df_final_demo.txt', delimiter=',')
#df_final_experiment_clients = pd.read_csv('df_final_experiment_clients.txt', delimiter=',')
#df_final_web_data_pt_1 = pd.read_csv('df_final_web_data_pt_1.txt', delimiter=',')
#df_final_web_data_pt_2 = pd.read_csv('df_final_web_data_pt_2.txt', delimiter=',')

In [ ]:
'''# Check for missing values
print(df_final_demo.isnull().sum())

# Fill missing values or drop rows with significant missing data
df_final_demo['clnt_age'] = df_final_demo['clnt_age'].fillna(df_final_demo['clnt_age'].median())
df_final_demo['gendr'] = df_final_demo['gendr'].fillna('Unknown')

# Replace negative or invalid values with NaN and handle them
df_final_demo['bal'] = df_final_demo['bal'].apply(lambda x: np.nan if x < 0 else x)
df_final_demo = df_final_demo.dropna()  # Drop rows with invalid values
'''

In [4]:
#test case
#print(df_final_experiment_clients.head())

In [ ]:
'''
#clean df_final_experiment
if 'Variation' in df_final_experiment_clients.columns:
    print(df_final_experiment_clients['Variation'].unique())
else:
    print("'Variation' column not found. Cannot proceed with analysis.")
'''

In [ ]:
'''
df_final_experiment_clients = pd.read_csv('df_final_experiment_clients.csv', delimiter=',')

# Ensure Variation contains only valid entries
print(df_final_experiment_clients['Variation'].unique())

# Fix invalid variations (if any)
valid_variations = ['Test', 'Control']
df_final_experiment_clients = df_final_experiment_clients[
    df_final_experiment_clients['Variation'].isin(valid_variations)
]'''

In [ ]:
'''
# Concatenate the two parts
df_final_web_data = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)

# Convert `date_time` to datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# Check for duplicates and drop them
df_final_web_data = df_final_web_data.drop_duplicates()

# Ensure `process_step` is valid (e.g., integers or known steps)
print(df_final_web_data['process_step'].unique())
'''

In [ ]:
'''
merged_data = pd.merge(df_final_demo, df_final_experiment_clients, on='client_id', how='inner')

# Merge the web data
merged_data = pd.merge(merged_data, df_final_web_data, on='client_id', how='inner')
'''

In [9]:
#merged_data.to_csv('cleaned_data.csv', index=False)


In [10]:
#print("Current Working Directory:", os.getcwd())
#os.chdir('..vanguard-ab-test/data/clean')

In [ ]:
#print out csv header 
df_all_clean = pd.read_csv("../cleaned_data.csv", dtype={'Variation': 'str'}, low_memory=False)
print(df_all_clean.dtypes)

In [ ]:
unique_count = df_all_clean['client_id'].nunique()
print(f"Number of unique client IDs: {unique_count}")


In [ ]:
# Check the structure of the data
print(df_all_clean.info())
print(df_all_clean.head())

In [ ]:
print(df_all_clean.describe())
print(df_all_clean['gendr'].value_counts())

In [ ]:
# Set pastel color for axis labels
pastel_color = '#AEC6CF'  # Example pastel blue from a pastel palette

# Plot with pastel style
df_all_clean['clnt_age'].hist(bins=20, color='#5F9EA0', edgecolor='white')  # Add pastel orange for the bars
plt.title('Age Distribution', color='#00CED1')  # Pastel green title
plt.xlabel('Age', color='#008B8B')  # Use pastel blue for the label
plt.ylabel('Frequency', color='#20B2AA')  # Use pastel blue for the label
#plt.savefig('average_age_by_variation.png', dpi=300, bbox_inches='tight')  # Save as PNG
plt.show()



In [ ]:
#All test views
print(df_all_clean.groupby('Variation')['clnt_age'].mean())
print(df_all_clean.groupby('Variation')['bal'].sum())
print(df_all_clean['gendr'].value_counts())



In [ ]:
if {'client_id', 'gendr'}.issubset(data.columns):
    # Get unique clients
    unique_clients = data.drop_duplicates(subset='client_id')

    # Filter for 'F' and 'M' genders
    filtered_clients = unique_clients[unique_clients['gendr'].isin(['F', 'M'])]

    # Count clients grouped by gender
    gender_counts = filtered_clients['gendr'].value_counts()

    # Calculate percentages
    gender_percentages = (gender_counts / gender_counts.sum()) * 100

    # Combine counts and percentages into a DataFrame for clarity
    gender_stats = pd.DataFrame({
        'Gender': gender_counts.index,
        'Client Count': gender_counts.values,
        'Percentage': gender_percentages.values
    })

    # Display results
    print(gender_stats)

In [ ]:
#Time spend in process steps
df_all_clean['date_time'] = pd.to_datetime(df_all_clean['date_time'])

# Sort data by client and timestamp
df_all_clean = df_all_clean.sort_values(by=['client_id', 'date_time'])

# Calculate time differences
df_all_clean['time_diff'] = df_all_clean.groupby('client_id')['date_time'].diff()

# Average time spent per step
time_per_step = df_all_clean.groupby('process_step')['time_diff'].mean()
print(time_per_step)


In [ ]:
# Filter for the 'confirm' step
completion_data = df_all_clean[df_all_clean['process_step'] == 'confirm']

# Calculate completion rates
total_test = len(df_all_clean[df_all_clean['Variation'] == 'Test'])
total_control = len(df_all_clean[df_all_clean['Variation'] == 'Control'])
confirm_test = len(completion_data[completion_data['Variation'] == 'Test'])
confirm_control = len(completion_data[completion_data['Variation'] == 'Control'])

test_completion_rate = confirm_test / total_test
control_completion_rate = confirm_control / total_control

print(f"Test Completion Rate: {test_completion_rate}")
print(f"Control Completion Rate: {control_completion_rate}")


In [19]:
data = pd.read_csv("../data/clean/clean_data_v1.csv", dtype={'Variation': 'str'}, low_memory=False)

In [ ]:
# Convert date_time column to datetime type for calculations
data['date_time'] = pd.to_datetime(data['date_time'])

# Completion Rate Calculation
# Filter data for 'confirm' step
confirm_data = data[data['process_step'] == 'confirm']

# Total users in Test and Control groups
test_users = data[data['Variation'] == 'Test']['visitor_id'].nunique()
control_users = data[data['Variation'] == 'Control']['visitor_id'].nunique()

# Users who reached the 'confirm' step in each group
test_confirm_users = confirm_data[confirm_data['Variation'] == 'Test']['visitor_id'].nunique()
control_confirm_users = confirm_data[confirm_data['Variation'] == 'Control']['visitor_id'].nunique()

# Completion rates
test_completion_rate = test_confirm_users / test_users
control_completion_rate = control_confirm_users / control_users

# Time Spent on Each Step Calculation
# Calculate time difference between consecutive steps for each visit
data = data.sort_values(by=['visitor_id', 'date_time'])
data['time_diff'] = data.groupby('visitor_id')['date_time'].diff()

# Average time spent on each step by group
time_spent_by_step = data.groupby(['Variation', 'process_step'])['time_diff'].mean().reset_index()

# Error Rate Calculation
# Determine backward navigation (process_step decreases)
data['step_number'] = data['process_step'].str.extract('(\d+)').astype(float)
data['step_diff'] = data.groupby('visitor_id')['step_number'].diff()

# Identify backward steps
data['is_backward'] = data['step_diff'] < 0

# Error rate for Test and Control groups
test_error_rate = data[data['Variation'] == 'Test']['is_backward'].sum() / len(data[data['Variation'] == 'Test'])
control_error_rate = data[data['Variation'] == 'Control']['is_backward'].sum() / len(data[data['Variation'] == 'Control'])

# Display the results
metrics = {
    "Test Completion Rate": f"{test_completion_rate:.2%}",
    "Control Completion Rate": f"{control_completion_rate:.2%}",
    "Test Error Rate": f"{test_error_rate:.2%}",
    "Control Error Rate": f"{control_error_rate:.2%}"
}

time_spent_by_step, metrics


In [39]:
plt.figure(figsize=(12, 6))
plt.plot(test_age_groups['clnt_age'], test_age_groups['error_rate'], label='Test Group', color='blue', marker='o')
plt.plot(control_age_groups['clnt_age'], control_age_groups['error_rate'], label='Control Group', color='red', marker='o')
plt.xlabel('Age', fontsize=14)
plt.ylabel('Error Rate', fontsize=14)
plt.title('Error Rate by Age Group for Test and Control Groups', fontsize=16)
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
#plt.savefig('Error Rate by Age Group for Test and Control Groups', format='png', dpi=300)
plt.show()


NameError: name 'test_age_groups' is not defined

<Figure size 1200x600 with 0 Axes>

In [54]:
# Successes and samples
successes = [confirm_test, confirm_control]
samples = [total_test, total_control]

# Perform z-test
z_stat, p_value = proportions_ztest(successes, samples)
print(f"Z-Statistic: {z_stat:.2f}, P-Value: {p_value:.4f}")

# Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference in completion rates.")
else:
    print("Fail to reject the null hypothesis: No significant difference in completion rates.")


Z-Statistic: 17.60, P-Value: 0.0000
Reject the null hypothesis: There is a significant difference in completion rates.


In [53]:
# Adjust threshold for one-sided test
threshold = 0.05  # 5% increase required

# Recalculate z-test with adjusted threshold
from statsmodels.stats.proportion import proportions_ztest

# Modify success array to reflect threshold
successes_adjusted = [confirm_test, confirm_control]
samples_adjusted = [total_test, total_control]

# Perform one-sided z-test
z_stat_threshold, p_value_threshold = proportions_ztest(successes_adjusted, samples_adjusted, alternative='larger')
observed_increase = test_completion_rate - control_completion_rate

print(f"Observed Increase: {observed_increase:.2%}")
print(f"Z-Statistic (Threshold Test): {z_stat_threshold:.2f}, P-Value: {p_value_threshold:.4f}")

# Conclusion
if observed_increase >= threshold and p_value_threshold < 0.05:
    print("Reject the null hypothesis: The Test group meets the 5% threshold.")
else:
    print("Fail to reject the null hypothesis: The Test group does not meet the 5% threshold.")


Observed Increase: 5.97%
Z-Statistic (Threshold Test): 17.60, P-Value: 0.0000
Reject the null hypothesis: The Test group meets the 5% threshold.


In [ ]:
# Data for visualization
groups = ['Test Group', 'Control Group']
completion_rates = [test_completion_rate * 100, control_completion_rate * 100]  # Convert to percentage
colors = ['skyblue', 'lightcoral']

# Fun visualization: Completion rate as donuts
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(aspect="equal"))

# Define the labels with percentages
labels = [f"{group}\n{rate:.2f}%" for group, rate in zip(groups, completion_rates)]

# Create a pie chart
wedges, texts = ax.pie(completion_rates, colors=colors, startangle=140, wedgeprops=dict(width=0.3, edgecolor='w'))

# Add annotations to the wedges
bbox_props = dict(boxstyle="square,pad=0.3", fc="white", ec="black", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1) / 2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(labels[i], xy=(x, y), xytext=(1.5 * np.sign(x), 1.5 * y),
                horizontalalignment=horizontalalignment, **kw)

# Add title
plt.title("Completion Rates: Test vs Control", fontsize=16)

# Save and display the fun visualization
#plt.savefig('fun_completion_rate_donut.png', format='png', dpi=300)
plt.show()




In [28]:
# Check if necessary columns are present
if 'clnt_age' in data.columns and 'clnt_tenure_yr' in data.columns and 'clnt_tenure_mnth' in data.columns:
    # Calculate total tenure in months
    data['total_tenure_months'] = data['clnt_tenure_yr'] * 12 + data['clnt_tenure_mnth']

    # Group by age and calculate average and total tenure
    tenure_by_age = data.groupby('clnt_age')['total_tenure_months'].agg(
        average_tenure='mean',
        total_tenure='sum'
    ).reset_index()

    # Calculate overall total tenure
    total_tenure = data['total_tenure_months'].sum()

    # Display results
    tenure_by_age, total_tenure
else:
    "The required columns ('clnt_age', 'clnt_tenure_yr', 'clnt_tenure_mnth') are not present in the dataset."


In [38]:
# Load data
data = pd.read_csv("../data/clean/clean_data_v1.csv", dtype={'Variation': 'str'}, low_memory=False)

# Ensure necessary columns are present
if {'client_id', 'gendr', 'bal'}.issubset(data.columns):
    # Get unique clients
    unique_clients = data.drop_duplicates(subset='client_id')

    # Select only relevant columns
    unique_client_data = unique_clients[['client_id', 'gendr', 'bal']]

    # Save the data locally
    unique_client_data.to_csv('unique_clients_with_gender_and_balance.csv', index=False)
    print("File saved as 'unique_clients_with_gender_and_balance.csv'")
else:
    print("The required columns ('client_id', 'gendr', 'balance') are not present in the dataset.")


File saved as 'unique_clients_with_gender_and_balance.csv'


In [52]:
# Check if necessary columns are present
if {'client_id', 'clnt_age', 'process_step', 'Variation'}.issubset(data.columns):
    # Map process_step to numerical values (assuming sequential step names like "Step 1")
    data['step_number'] = data['process_step'].str.extract(r'(\d+)').astype(float)

    # Sort by client_id and time to ensure proper sequence
    data = data.sort_values(by=['client_id', 'date_time'])

    # Calculate step differences
    data['step_diff'] = data.groupby('client_id')['step_number'].diff()

    # Identify backward steps
    data['is_backward'] = data['step_diff'] < 0

    # Define age brackets
    bins = [0, 18, 25, 35, 45, 55, 65, 100]
    labels = ['<18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
    data['age_bracket'] = pd.cut(data['clnt_age'], bins=bins, labels=labels, right=False)

    # Calculate error rates for each age bracket and group by Variation
    error_rate_by_age = data.groupby(['age_bracket', 'Variation']).agg(
        total_steps=('client_id', 'count'),
        backward_steps=('is_backward', 'sum')
    ).reset_index()
    error_rate_by_age['error_rate'] = error_rate_by_age['backward_steps'] / error_rate_by_age['total_steps']

    # Print the results
    print("Error Rates by Age Bracket and Variation Group:")
    print(error_rate_by_age)

else:
    print("The required columns ('client_id', 'clnt_age', 'process_step', 'Variation') are not present in the dataset.")


Error Rates by Age Bracket and Variation Group:
   age_bracket Variation  total_steps  backward_steps  error_rate
0          <18   Control          190               4    0.021053
1          <18      Test          240               6    0.025000
2        18-24   Control         8539             365    0.042745
3        18-24      Test        10688             219    0.020490
4        25-34   Control        27346             818    0.029913
5        25-34      Test        33076             565    0.017082
6        35-44   Control        22308             644    0.028869
7        35-44      Test        27148             556    0.020480
8        45-54   Control        27456             863    0.031432
9        45-54      Test        34733            1017    0.029281
10       55-64   Control        32244            1107    0.034332
11       55-64      Test        41421            1748    0.042201
12         65+   Control        22399             811    0.036207
13         65+      Test    

C:\Users\fabia\AppData\Local\Temp\ipykernel_16968\3182870779.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  error_rate_by_age = data.groupby(['age_bracket', 'Variation']).agg(
